### Callback API
- 모델이 학습 중에 충돌이 발생하거나 네트워크가 끊기면 모든 훈련 시간이 낭비될 수 있고,  
  과적합을 방지하기 위해 훈련을 조기 종료해야 할 수도 있다.
- 모델이 학습을 시작하면 학습이 완료될 때까지 어떤 제어도 하지 못하게 되고,  
  신경망 훈련을 완료하는 데에는 몇 시간에서 최대 며칠이 걸릴 수도 있기 때문에, 모델을 모니터링 및 제어하는 기능이 필요하다.
- 훈련 시(fit()) Callback API를 등록시키면 반복 내에서 특정 이벤트가 발생할 때마다 등록된 callback이 호출되어 수행된다.

<br>

**ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weight_only=False, mode='auto')**

- 특정 조건에 따라 모델 또는 가중치를 파일로 저장한다.
- filepath: "weights.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.weights.hdf5"와 같이 모델의 체크포인트를 저장한다.
- monitor: 모니터링할 성능 지표를 작성한다.
- save_best_only: 가장 좋은 성능을 보인 모델을 저장할지에 대한 여부
- mode: {auto, min, max} 중 한 가지를 작성한다. monitor의 성능 지표에 따라 적합한 것을 선택한다.  
  monitor의 성능 지표가 감소해야 좋은 경우는 min, 증가해야 좋은 경우는 max,  
  monitor의 성능 지표명으로부터 자동으로 유추하고 싶다면 auto를 기재한다.

<br>

**ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', min_lr=0)**

- 특정 반복 동안 성능이 개선되지 않을 때, 학습률을 동적으로 감소시킨다.
- monitor: 모니터링할 성능 지표를 작성한다.
- factor: 학습률을 감소시킬 비율, 새로운 학습률 = 기존 학습률 * factor
- patience: 학습률을 줄이기 전 monitor 할 반복 횟수
- mode: {auto, min, max} 중 한 가지를 작성한다. monitor의 성능 지표에 따라 적합한 것을 선택한다.  
  monitor의 성능 지표가 감소해야 좋은 경우는 min, 증가해야 좋은 경우는 max,  
  monitor의 성능 지표명으로부터 자동으로 유추하고 싶다면 auto를 기재한다.

<br>

**EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')**

- 특정 반복동안 성능이 개선되지 않으면 학습을 조기 종료한다.
- monitor: 모니터링할 성능 지표를 작성한다.
- patience: Early Stopping을 적용하기 전 monitor할 반복 횟수
- mode: {auto, min, max} 중 한 가지를 작성한다. monitor의 성능 지표에 따라 적합한 것을 선택한다.  
  monitor의 성능 지표가 감소해야 좋은 경우는 min, 증가해야 좋은 경우는 max,  
  monitor의 성능 지표명으로부터 자동으로 유추하고 싶다면 auto를 기재한다.

In [12]:
from tensorflow.keras.layers import Layer, Input, Flatten, Dense
from tensorflow.keras.models import Model

INPUT_SIZE = 28

# 모델 생성 함수 선언
def create_model():
    input_tensor = Input(shape=(INPUT_SIZE, INPUT_SIZE))

    x = Flatten()(input_tensor)

    x = Dense(64, activation='relu')(x)
    x = Dense(128, activation='relu')(x)

    output = Dense(10, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=output)
    
    return model

In [13]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np

# images MinMaxScaling 함수 선언
def get_preprocessed_data(images, targets):
    images = np.array(images / 255.0, dtype=np.float32)
    targets = np.array(targets, dtype=np.float32)

    return images, targets

# targets 원핫 인코딩 함수 선언 (+ images MinMaxScaling)
def get_preprocessed_ohe(images, targets):
    images, targets = get_preprocessed_data(images, targets)
    oh_targets = to_categorical(targets)

    return images, oh_targets

# train 데이터에서 validation 데이터 분리하는 함수 선언 (+ images MinMaxScaling, targets 원핫 인코딩)
def get_train_valid_test(train_images, train_targets, test_images, test_targets, validation_size=0.2, random_state=124):
    train_images, train_oh_targets = get_preprocessed_ohe(train_images, train_targets)
    test_images, test_oh_targets = get_preprocessed_ohe(test_images, test_targets)

    train_images, validation_images, train_oh_targets, validation_oh_targets = \
    train_test_split(train_images, train_oh_targets, stratify=train_oh_targets, test_size=validation_size, random_state=random_state)

    return (train_images, train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_oh_targets)

In [14]:
from tensorflow.keras.datasets import fashion_mnist

# fashion 데이터 세트 불러오기
(train_images, train_targets), (test_images, test_targets) = fashion_mnist.load_data()

# 위에서 선언한 함수로 스케일링, 원핫 인코딩, validation 데이터 분리
(train_images, train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_oh_targets) = \
                                            get_train_valid_test(train_images, train_targets, test_images, test_targets)

# train, validation, test 데이터 shape 출력
print(train_images.shape, train_oh_targets.shape)
print(validation_images.shape, validation_oh_targets.shape)
print(test_images.shape, test_oh_targets.shape)

(48000, 28, 28) (48000, 10)
(12000, 28, 28) (12000, 10)
(10000, 28, 28) (10000, 10)


In [15]:
# callback_files 경로 표시
!dir callback_files

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: ED2A-EA84

 C:\KDT_0900_KGH\ai\deep_learning\c_tensorflow\callback_files 디렉터리

2024-05-27  오후 01:25    <DIR>          .
2024-05-28  오전 10:44    <DIR>          ..
               0개 파일                   0 바이트
               2개 디렉터리  101,164,036,096 바이트 남음


#### ModelCheckpoint

In [16]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint

# 모델 컴파일링
model = create_model()
model.compile(optimizer=Adam(0.001), loss=CategoricalCrossentropy(), metrics=['acc'])

# callback (ModelCheckpoint) 선언 - 가중치(weight)만 저장 (확장자 .weights.h5)
mcp_cb = ModelCheckpoint(
    filepath='./callback_files/weights.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.weights.h5',
    monitor='val_loss',

    # save_best_only=True: 모든 epoch의 파일을 저장하지 않고, 성능이 좋다고 판단했을 경우에만 저장
    save_best_only=False,
    save_weights_only=True,
    mode='min'
)

# save_weights_only=False: 가중치가 아닌 모델을 저장 (확장자 .model.keras)
# mcp_cb = ModelCheckpoint(
#     filepath="./callback_files/model.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.model.keras",
#     monitor='val_loss',
#     save_best_only=False,
#     save_weights_only=False,
#     mode='min'
# )

# 모델 훈련과 동시에 train, validation 데이터에 대한 정확도와 loss 이력 저장
history = model.fit(x=train_images, 
                    y=train_oh_targets, 
                    batch_size=64, 
                    epochs=20, 
                    validation_data=(validation_images, validation_oh_targets), callbacks=[mcp_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - acc: 0.7492 - loss: 0.7346 - val_acc: 0.8373 - val_loss: 0.4578
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8546 - loss: 0.4078 - val_acc: 0.8552 - val_loss: 0.3948
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.8692 - loss: 0.3611 - val_acc: 0.8621 - val_loss: 0.3636
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.8783 - loss: 0.3321 - val_acc: 0.8690 - val_loss: 0.3474
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.8861 - loss: 0.3123 - val_acc: 0.8732 - val_loss: 0.3414
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8878 - loss: 0.3011 - val_acc: 0.8760 - val_loss: 0.3314
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.8946 - loss: 0.2815 - val_acc: 0.8745 - val_loss: 0.3476
Epoch 8/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8976 - loss: 0.2676 - val_acc: 0.8818 - val_loss: 0.3144
Epoch 9/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - ac

In [17]:
# callback_files 경로 표시
!dir callback_files

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: ED2A-EA84

 C:\KDT_0900_KGH\ai\deep_learning\c_tensorflow\callback_files 디렉터리

2024-05-28  오전 10:50    <DIR>          .
2024-05-28  오전 10:50    <DIR>          ..
2024-05-28  오전 10:50           739,600 weights.001-0.4578-0.8099.weights.h5
2024-05-28  오전 10:50           739,600 weights.002-0.3948-0.8577.weights.h5
2024-05-28  오전 10:50           739,600 weights.003-0.3636-0.8719.weights.h5
2024-05-28  오전 10:50           739,600 weights.004-0.3474-0.8788.weights.h5
2024-05-28  오전 10:50           739,600 weights.005-0.3414-0.8856.weights.h5
2024-05-28  오전 10:50           739,600 weights.006-0.3314-0.8888.weights.h5
2024-05-28  오전 10:50           739,600 weights.007-0.3476-0.8945.weights.h5
2024-05-28  오전 10:50           739,600 weights.008-0.3144-0.8983.weights.h5
2024-05-28  오전 10:50           739,600 weights.009-0.3119-0.9025.weights.h5
2024-05-28  오전 10:50           739,600 weights.010-0.3137-0.9065.weights.h5
2024-05-28  오전 10:50           739,600 weig

In [18]:
# 현재 모델 성능 검증
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 868us/step - acc: 0.8857 - loss: 0.3469


[0.34664174914360046, 0.885699987411499]

In [19]:
# 새로운 모델 생성 후 미리 설정한 가중치 부여
model = create_model()
model.load_weights('./callback_files/weights.020-0.3276-0.9311.weights.h5')

# 가중치 설정한 모델로 성능 검증 - 같은 모델에 가중치도 같으니 결과 역시 같다
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 959us/step - acc: 0.8857 - loss: 0.3469


[0.34664174914360046, 0.885699987411499]

In [ ]:
# 모델 자체를 내보냈을 경우, 아래처럼 모델 파일 불러온 다음 바로 사용하면 된다
# from tensorflow.keras.models import load_model

# model = load_model('./callback_files/model.020-0.3350-0.9287.model.keras')
# model.evaluate(test_images, test_oh_targets)

#### ReduceLROnPlateau

In [7]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau

# 모델 컴파일링
model = create_model()
model.compile(optimizer=Adam(0.001), loss=CategoricalCrossentropy(), metrics=['acc'])

# callback (ReduceLROnPlateau) 선언
rlr_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2,
    mode='min'
)

# 모델 훈련과 동시에 train, validation 데이터에 대한 정확도와 loss 이력 저장
history = model.fit(x=train_images, 
                    y=train_oh_targets, 
                    batch_size=64, 
                    epochs=20, 
                    validation_data=(validation_images, validation_oh_targets), callbacks=[rlr_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - acc: 0.7408 - loss: 0.7531 - val_acc: 0.8527 - val_loss: 0.4087 - learning_rate: 0.0010
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8583 - loss: 0.3960 - val_acc: 0.8502 - val_loss: 0.4005 - learning_rate: 0.0010
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8711 - loss: 0.3604 - val_acc: 0.8644 - val_loss: 0.3624 - learning_rate: 0.0010
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.8776 - loss: 0.3301 - val_acc: 0.8734 - val_loss: 0.3537 - learning_rate: 0.0010
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.8876 - loss: 0.3097 - val_acc: 0.8673 - val_loss: 0.3567 - learning_rate: 0.0010
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.8900 - loss: 0.2969 - val_acc: 0.8771 - val_loss: 0.3310 - learning_rate: 0.0010
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.8955 - loss: 0.2814 - val_acc: 0.8814 - val_loss: 0.3228 - learning_rate: 0.0010
Epoch 

#### EarlyStopping

In [8]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

# 모델 컴파일링
model = create_model()
model.compile(optimizer=Adam(0.001), loss=CategoricalCrossentropy(), metrics=['acc'])

# callback (EarlyStopping) 선언
ely_cb = EarlyStopping(
    monitor='val_loss',
    patience=3,
    mode='min'
)

# 모델 훈련과 동시에 train, validation 데이터에 대한 정확도와 loss 이력 저장
history = model.fit(x=train_images, 
                    y=train_oh_targets, 
                    batch_size=64, 
                    epochs=20, 
                    validation_data=(validation_images, validation_oh_targets), callbacks=[ely_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.7299 - loss: 0.7767 - val_acc: 0.8461 - val_loss: 0.4236
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8518 - loss: 0.4123 - val_acc: 0.8565 - val_loss: 0.3979
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.8665 - loss: 0.3713 - val_acc: 0.8645 - val_loss: 0.3662
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.8756 - loss: 0.3377 - val_acc: 0.8753 - val_loss: 0.3492
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.8851 - loss: 0.3127 - val_acc: 0.8648 - val_loss: 0.3655
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.8897 - loss: 0.2963 - val_acc: 0.8772 - val_loss: 0.3384
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8923 - loss: 0.2879 - val_acc: 0.8789 - val_loss: 0.3240
Epoch 8/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.8986 - loss: 0.2761 - val_acc: 0.8808 - val_loss: 0.3183
Epoch 9/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - ac

In [20]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# 모델 컴파일링
model = create_model()
model.compile(optimizer=Adam(0.001), loss=CategoricalCrossentropy(), metrics=['acc'])

# callback (ModelCheckpoint) 선언 - 가중치(weight)만 저장
mcp_cb = ModelCheckpoint(
    filepath='./callback_files/weights.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.weights.h5',
    monitor='val_loss',

    # save_best_only=True: 모든 epoch의 파일을 저장하지 않고, 성능이 좋다고 판단했을 경우에만 저장
    save_best_only=False,
    save_weights_only=True,
    mode='min'
)

# callback (ReduceLROnPlateau) 선언
rlr_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2,
    mode='min'
)

# callback (EarlyStopping) 선언
ely_cb = EarlyStopping(
    monitor='val_loss',
    patience=3,
    mode='min'
)

# 모델 훈련과 동시에 train, validation 데이터에 대한 정확도와 loss 이력 저장
history = model.fit(x=train_images, 
                    y=train_oh_targets, 
                    batch_size=64, 
                    epochs=20, 
                    validation_data=(validation_images, validation_oh_targets), callbacks=[mcp_cb, rlr_cb, ely_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.7323 - loss: 0.7640 - val_acc: 0.8508 - val_loss: 0.4114 - learning_rate: 0.0010
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8552 - loss: 0.4021 - val_acc: 0.8583 - val_loss: 0.3867 - learning_rate: 0.0010
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.8711 - loss: 0.3562 - val_acc: 0.8640 - val_loss: 0.3673 - learning_rate: 0.0010
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8817 - loss: 0.3250 - val_acc: 0.8717 - val_loss: 0.3414 - learning_rate: 0.0010
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8871 - loss: 0.3084 - val_acc: 0.8695 - val_loss: 0.3482 - learning_rate: 0.0010
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8928 - loss: 0.2920 - val_acc: 0.8748 - val_loss: 0.3381 - learning_rate: 0.0010
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.8953 - loss: 0.2815 - val_acc: 0.8774 - val_loss: 0.3306 - learning_rate: 0.0010
Epoch 